In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
])
train_dataset = torchvision.datasets.MNIST('~/.torch/models/mnist', train=True, download=True, 
                                           transform=transform)
test_dataset = torchvision.datasets.MNIST('~/.torch/models/mnist', train=False, download=True, 
                                           transform=transform)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=True)

Processing...
Done!


In [3]:
def assert_shape(x, shape):
    assert tuple(x.shape[-2:]) == tuple(shape), f'Expected shape ending {shape}, got {x.shape}'

In [34]:
class Encoder(nn.Module):
    def __init__(self, size: (int, int)):
        super().__init__()
        self.size = size
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=25, kernel_size=12, padding=0, stride=2)
        self.conv2 = nn.Conv2d(in_channels=25, out_channels=64, kernel_size=5, padding=2)

    
    def forward(self, x):
        assert_shape(x, self.size)
        
        x = F.relu(self.conv1(x))
        assert_shape(x, (9, 9)) # Should this be the same size?

        x = F.relu(self.conv2(x))
        assert_shape(x, (9, 9))
        
        x = F.max_pool2d(x, 2)
        assert_shape(x, (4, 4))
        
        return x
    
class Classifier(nn.Module):
    def __init__(self, num_classes: int):
        super().__init__()
        self.fc1 = nn.Linear(in_features=4*4*64, out_features=1024)
        self.fc2 = nn.Linear(in_features=1024, out_features=num_classes)
        
    def forward(self, x):
        assert_shape(x, (4, 4))
        
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 16, 3, stride=2),  # b, 16, 5, 5
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 5, stride=3, padding=0),  # b, 8, 15, 15
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 2, stride=1, padding=1),  # b, 1, 28, 28
            nn.Tanh()
        )

    def forward(self, x):
        x = self.decoder(x)
        return x
    
class Model(nn.Module):
    def __init__(self, size: (int, int), num_classes: int):
        super().__init__()
        self.size = size
        self.encoder = Encoder(size)
        self.autoencoder = Decoder()
        self.decoder = Classifier(num_classes=10)
        
        self.weight1 = nn.Parameter(torch.tensor([1.0]))
        self.weight2 = nn.Parameter(torch.tensor([1.0]))
        
    def forward(self, x):
        assert_shape(x, self.size)
#         print(x.size())
        x = self.encoder(x)
#         print(x.size())
        cls = self.decoder(x)
#         print(cls.size())
        gen = self.autoencoder(x)
#         print(gen.size())
        return cls, gen
        
model = Model((28,28), 10)
model = model
for image, labels in train_dataloader:
    image = image
    image /= 255
    model(image)
    break

In [35]:
criterion1 = nn.CrossEntropyLoss()
criterion2 = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

enable1 = True
enable2 = True
learn_weights = True

In [36]:
for epoch in range(10):
    epoch_loss = 0
    for i, data in enumerate(train_dataloader):
        image, labels = data
        
        image = image
        labels = labels
        
        image /= 255
        
        optimizer.zero_grad()
        
        output1, output2 = model(image)
        
        if enable1:
            loss1 = criterion1(output1, labels)
        else:
            loss1 = 0
            
        if enable2:
            loss2 = criterion2(image, output2)
        else:
            loss2 = 0
        
        if learn_weights:
            loss = (torch.exp(-model.weight1) * loss1 + 0.5 * model.weight1 
                    + 0.5 * (torch.exp(-model.weight2) * loss2 + model.weight2))
        else:
            loss = loss1 + loss2
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        
    print(f'Epoch {epoch}: {epoch_loss/i:.3} ({model.weight1.item()}, {model.weight2.item()})')

KeyboardInterrupt: 

In [19]:
with torch.no_grad():
    correct1 = 0
    correct2 = 0
    total = 0
    
    for i, data in enumerate(test_dataloader):
        image, labels = data
        
        image = image.cuda()
        labels = labels.cuda()
        
        output1, output2 = model(image)
        preds1 = output1.argmax(dim=1)
        preds2 = output2.argmax(dim=1)
        
        correct1 += (labels_to_1(labels) == preds1).sum().item()
        correct2 += (labels == preds2).sum().item()
        total += preds1.shape[0]
    
    print(f'Accuracy 1: {correct1}/{total} ({100 * correct1/total:.2f}%)')
    print(f'Accuracy 2: {correct2}/{total} ({100 * correct2/total:.2f}%)')

Accuracy 1: 9947/10000 (99.47%)
Accuracy 2: 9853/10000 (98.53%)


Just 1:
Accuracy 1: 9868/10000 (98.68%)
Accuracy 2: 1352/10000 (13.52%)

Just 2:
Accuracy 1: 1270/10000 (12.70%)
Accuracy 2: 9434/10000 (94.34%)

Both (equal weights):
Accuracy 1: 9840/10000 (98.40%)
Accuracy 2: 9457/10000 (94.57%)

Both (learned weights):
Accuracy 1: 9897/10000 (98.97%)
Accuracy 2: 9735/10000 (97.35%)